The goal of this notebook is to model serve via vLLM the Large MoE model Qwen3 235B A22B within Databricks (as an all purpose cluster). 
- The original author of this notebook is [Srinivas Billa](https://github.com/nivibilla).
- Requires an instance type of g6e.48xlarge or larger to run.
- Future goal: To get serverless involved in serving large MoE models such as Qwen3 (not only PaaS)

In [0]:
!pip install uv
dbutils.library.restartPython()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/21.1 MB ? eta -:--:--
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.4/21.1 MB 11.1 MB/s eta 0:00:02
   ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/21.1 MB 13.8 MB/s eta 0:00:02
   ━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/21.1 MB 16.2 MB/s eta 0:00:02
   ━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/21.1 MB 17.6 MB/s eta 0:00:02
   ━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/21.1 MB 20.0 MB/s eta 0:00:01
   ━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/21.1 MB 23.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/21.1 MB 26.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━ 8.8/21.1 MB 31.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━ 11.7/21.1 MB 47.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━ 15.5/21.1 MB 78.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━ 20.5/21.1 MB 119.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 21.1/21.1 MB 121.3 MB/s 

In [0]:
!uv pip install huggingface_hub
!uv pip install "vllm==0.10.1.1"

Using Python 3.12.3 environment at: /local_disk0/.ephemeral_nfs/envs/pythonEnv-edf3d31a-82bd-49bb-9289-567175bcca4f
Resolved 13 packages in 115ms
⠙ Preparing packages... (0/13)
⠙ Preparing packages... (0/13)
⠙ Preparing packages... (0/13)
filelock             ------------------------------     0 B/15.61 KiB
⠙ Preparing packages... (0/13)
filelock             ------------------------------     0 B/15.61 KiB
⠙ Preparing packages... (0/13)
filelock             ------------------------------     0 B/15.61 KiB
⠙ Preparing packages... (0/13)
filelock             ------------------------------     0 B/15.61 KiB
requests             ------------------------------     0 B/63.22 KiB
⠙ Preparing packages... (0/13)
filelock             ------------------------------     0 B/15.61 KiB
requests             ------------------------------     0 B/63.22 KiB
⠙ Preparing packages... (0/13)
filelock             ------------------------------     0 B/15.61 KiB
requests             -------------------------

In [0]:
!pip uninstall numpy -y

Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:
import os
from huggingface_hub import snapshot_download

# os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

def dump_model_to_local(model_id, base_path="/local_disk0/models/", num_workers=4, ignore_patterns="*consolidated*"):

    print(snapshot_download(repo_id=model_id, local_dir=base_path + model_id, max_workers=num_workers, ignore_patterns=ignore_patterns))

dump_model_to_local("Qwen/Qwen3-235B-A22B-Instruct-2507-FP8")

# !mkdir -p /Volumes/sandbox/nbilla/testing/models/Qwen/Qwen3-0.6B/
# !cp -r /tmp/models/Qwen/Qwen3-0.6B /Volumes/sandbox/nbilla/testing/models/Qwen/

# # !mkdir -p /Volumes/sandbox/nbilla/testing/models/Qwen/Qwen3-0.6B/
# # !cp -r /tmp/models/Qwen/Qwen3-0.6B /Volumes/sandbox/nbilla/testing/models/Qwen/

Fetching 34 files:   0%|          | 0/34 [00:00<?, ?it/s]

LICENSE: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/local_disk0/models/Qwen/Qwen3-235B-A22B-Instruct-2507-FP8


In [0]:
!ls /local_disk0/models/Qwen/Qwen3-235B-A22B-Instruct-2507-FP8

LICENSE				  model-00013-of-00024.safetensors
README.md			  model-00014-of-00024.safetensors
config.json			  model-00015-of-00024.safetensors
generation_config.json		  model-00016-of-00024.safetensors
merges.txt			  model-00017-of-00024.safetensors
model-00001-of-00024.safetensors  model-00018-of-00024.safetensors
model-00002-of-00024.safetensors  model-00019-of-00024.safetensors
model-00003-of-00024.safetensors  model-00020-of-00024.safetensors
model-00004-of-00024.safetensors  model-00021-of-00024.safetensors
model-00005-of-00024.safetensors  model-00022-of-00024.safetensors
model-00006-of-00024.safetensors  model-00023-of-00024.safetensors
model-00007-of-00024.safetensors  model-00024-of-00024.safetensors
model-00008-of-00024.safetensors  model.safetensors.index.json
model-00009-of-00024.safetensors  tokenizer.json
model-00010-of-00024.safetensors  tokenizer_config.json
model-00011-of-00024.safetensors  vocab.json
model-00012-of-00024.safetensors


In [0]:
%restart_python

In [0]:
from dbruntime.databricks_repl_context import get_context
ctx = get_context()

port = "1111"
driver_proxy_api = f"https://{ctx.browserHostName}/driver-proxy-api/o/0/{ctx.clusterId}/{port}/v1"

print(f"""
driver_proxy_api = '{driver_proxy_api}'
cluster_id = '{ctx.clusterId}'
port = {port}
""")


driver_proxy_api = 'https://e2-demo-field-eng.cloud.databricks.com/driver-proxy-api/o/0/0915-194316-vqbbk5bl/1111/v1'
cluster_id = '0915-194316-vqbbk5bl'
port = 1111



In [0]:
# see https://docs.vllm.ai/en/stable/cli/serve.html for full options. (remove enforce eager for max thoughput and tune cuda graphs depending on memory available)
!vllm serve /local_disk0/models/Qwen/Qwen3-235B-A22B-Instruct-2507-FP8 --max-model-len 8192 --enforce-eager --port 1111 --host 0.0.0.0 --enable-expert-parallel --served-model-name Qwen3-235B --tensor-parallel-size 8

2025-09-16 19:08:56.669768: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-16 19:08:56.685518: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-16 19:08:56.690218: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-16 19:08:56.702370: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
AttributeError: 'MessageFactory' object has no attrib